## 从老版本的chain迁移到LCEL
***
- 示例1:从LLMChain迁移
- 示例2:从StuffDocumentsChain迁移

#### 从LLMChain迁移到LCEL

废弃的用法

In [1]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

legacy_chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)

legacy_result = legacy_chain({"adjective": "funny"})
legacy_result

/var/folders/gp/1nc10nrd3kjfxp5kkm9h93l80000gn/T/ipykernel_2820/1435414171.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  legacy_chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)
/var/folders/gp/1nc10nrd3kjfxp5kkm9h93l80000gn/T/ipykernel_2820/1435414171.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  legacy_result = legacy_chain({"adjective": "funny"})


{'adjective': 'funny',
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything!"}

LCEL的用法

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = prompt | ChatOpenAI() | StrOutputParser()

chain.invoke({"adjective": "funny"})

"Why couldn't the leopard play hide and seek?\n\nBecause he was always spotted!"

#### 从StuffDocumentsChain迁移

废弃的做法

In [4]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book"}),
]

from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
llm=ChatOpenAI()
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

/var/folders/gp/1nc10nrd3kjfxp5kkm9h93l80000gn/T/ipykernel_2820/3687773536.py:24: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  chain = StuffDocumentsChain(


In [5]:
result = chain.invoke(documents)
result["output_text"]

'The content describes the colors of different fruits: apples are red, blueberries are blue, and bananas are yellow.'

LCEL的做法

In [6]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book"}),
]

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
chain = create_stuff_documents_chain(llm, prompt)

result = chain.invoke({"context": documents})
result

'This content describes the colors of different fruits: apples are red, blueberries are blue, and bananas are yellow.'